In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
# load resnet for encoding image to match
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [3]:
# load PaddleOCR for text detection
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/03/07 17:47:06] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/03/07 17:47:06] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/03/07 17:47:06] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [4]:
# load operations
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.detect_text(paddle_ocr)
    op.shrink_target_element_img()
    op.resize(int(op.ui_img_width * (800 / op.ui_img_height)), 800)        
    operations.append(op)
#     op.show_target_ele()
#     break

# test with the first operation
# opr = operations[0]
# opr.show_target_ele()

[2022/03/07 17:47:06] root DEBUG: dt_boxes num : 1, elapse : 0.03921031951904297
[2022/03/07 17:47:06] root DEBUG: cls num  : 1, elapse : 0.004595756530761719
[2022/03/07 17:47:06] root DEBUG: rec_res num  : 1, elapse : 0.003875732421875
[Text Detection Completed in 0.051 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/07 17:47:06] root DEBUG: dt_boxes num : 3, elapse : 0.11318564414978027
[2022/03/07 17:47:06] root DEBUG: cls num  : 3, elapse : 0.011447668075561523
[2022/03/07 17:47:06] root DEBUG: rec_res num  : 3, elapse : 0.03669238090515137
[Text Detection Completed in 0.177 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/07 17:47:06] root DEBUG: dt_boxes num : 1, elapse : 0.03405618667602539
[2022/03/07 17:47:06] root DEBUG: cls num  : 1, elapse : 0.004242658615112305
[2022/03/07 17:47:06] root DEBUG: rec_res num  : 1, elapse : 0.004838228225708008
[Text Detection Completed in 0.051 s] Input: data/operation/target.pn

[2022/03/07 17:47:08] root DEBUG: cls num  : 0, elapse : 0
[2022/03/07 17:47:08] root DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[Text Detection Completed in 0.023 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/07 17:47:08] root DEBUG: dt_boxes num : 0, elapse : 0.011462926864624023
[2022/03/07 17:47:08] root DEBUG: cls num  : 0, elapse : 0
[2022/03/07 17:47:08] root DEBUG: rec_res num  : 0, elapse : 1.430511474609375e-06
[Text Detection Completed in 0.013 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/07 17:47:08] root DEBUG: dt_boxes num : 3, elapse : 0.09684205055236816
[2022/03/07 17:47:08] root DEBUG: cls num  : 3, elapse : 0.012254476547241211
[2022/03/07 17:47:08] root DEBUG: rec_res num  : 3, elapse : 0.035651445388793945
[Text Detection Completed in 0.162 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/03/07 17:47:08] root DEBUG: dt_boxes num : 1, elapse : 0.03572511672

In [5]:
# activate the robot
from robot.robot_control import RobotController
robot_1 = RobotController(port='/dev/ttyACM0')
robot_2 = RobotController(port='/dev/ttyACM1')

In [6]:
robot_y2img_x = round(125/540, 2)  # y_robot : x_img = 130: 540
robot_x2img_y = round((303-120)/820, 2)  # x_robot : y_img = 310: 820

def cvt_image_pos_to_robot_coord(x_img, y_img):
    x_robot = int((820 - y_img) * robot_x2img_y) + 120
    y_robot = int((270 - x_img) * robot_y2img_x)
    return x_robot, y_robot

In [7]:
robot_1.reset()
robot_2.reset()

[D 220307 17:47:28 robot_control:21] Loading Robot Drivers...
[D 220307 17:47:28 robot_control:21] Loading Robot Drivers...


In [2]:
# ********************************************
# *** Streaming Detection with Key Control ***
# ********************************************
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)

opr_id = 0
operate = False

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    frame = frame[80:900, :]
    
    cv2.imshow('camera', frame)
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
    elif key == ord('s'):
        if not operate:
            cv2.destroyWindow('element matching')
            opr = operations[opr_id]
            opr.show_target_ele(show=True)
            cv2.imwrite('data/output/temp.jpg', frame)

            # 1. detect GUI element in the frame         
            gui = GUI('data/output/temp.jpg')
            gui.detect_element(True, True, True, paddle_cor=paddle_ocr)

            # 2. focus on the screen region and adjust all elements
            gui.adjust_elements_by_screen()
            matched_ele = gui.match_elements(target_ele_img=opr.target_element_img, target_ele_text=opr.get_text_content(), resnet_model=resnet_model, min_similarity_img=0.55)

            # 3. show the result
            if matched_ele is not None:
                board = gui.img.copy()
                matched_ele.draw_element(board, show=False)
                board = cv2.hconcat([opr.show_target_ele(show=False), board, gui.det_result_imgs['merge']])
                cv2.imshow('element matching', board)
                operate = True
                opr_id += 1
                continue
            print('===================\n')
        
        else:
            # 4. operate the robot
            print('The position of matched element:(%d, %d)' % (matched_ele.center_x, matched_ele.center_y))
            x_robot, y_robot = cvt_image_pos_to_robot_coord(matched_ele.center_x, matched_ele.center_y)
            robot_1.click((x_robot, y_robot, 20))
            robot_2.click((x_robot, y_robot, 20))
            operate = False
            cv2.destroyWindow('element matching')
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()